
# Modeliranje i dekompozicija podataka

Iako algoritmi za automatsko modeliranje kao što je `auto_arima` mogu proizvesti modele koji predviđaju sa zadovoljavajućom preciznosti, važno je razumjeti parametre odabranog modela i usporediti ih sa stvarnim svojstvima odabranog skupa podataka.


In [ ]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd

from statsmodels.tsa.seasonal import STL

warnings.filterwarnings("ignore")

## Učitavanje i proširivanje podataka s ciljnom varijablom

Podatke razdvajamo u više skupova ovisno o kojem se hotelu radi (`hotel_id`) jer je očekivano da će se podaci za svaki od hotela ponašati različito i da će se predviđanje popunjenosti raditi za svaki hotel pojedinačno.

Ciljna varijabla je ukupan broj gostiju u nekom hotelu u nekom vremenskom periodu, gdje smo odredili da vremenski period može biti:
- dan
- tjedan
- mjesec
- godina

Čišćenje i proširivanje podataka radimo na način koji je opisan u bilježnici `ARIMA_occupancy_prediction.ipynb`.

In [ ]:
data = pd.read_parquet('../data/raw/export_df.parquet')
data.head()

In [ ]:
# Osnovno čišćenje podataka
data = data[~(data['zemlja_gosta']=='0')]
data['broj_djece_gostiju'] = data['broj_djece_gostiju'].astype(int)

# Ukupan broj gostiju po zapisu
data['ukupno_gostiju'] = data['broj_odraslih_gostiju'] + data['broj_djece_gostiju']

# Duljina boravka
data['duljina_boravka'] = data['datum_odjave'] - data['datum_dolaska']
data['duljina_boravka'] = data['duljina_boravka'].dt.days

# Rezervacije bez gostiju
data = data[data['ukupno_gostiju'] != 0]

# Otkazane rezervacije
data = data[~data['datum_otkazivanja_rezervacije'].notna()]
data = data.reset_index(drop=True)

# Razdvoji podatke iz različitih hotela
dataResort = data[data['hotel_id'] == 0]
dataCity = data[data['hotel_id'] == 1]

assert len(dataResort) + len(dataCity) == len(data)

print(f"Resort ima {len(dataResort)} zapisa")
print(f"City ima {len(dataCity)} zapisa")

In [ ]:
# Ciljne varijable - hotel 0
NumberOfGuestsResort = dataResort[['datum_dolaska', 'ukupno_gostiju']]

NumberOfGuestsDailyResort = dataResort['ukupno_gostiju'].groupby(dataResort['datum_dolaska']).sum()
NumberOfGuestsDailyResort = NumberOfGuestsDailyResort.resample('d').sum().to_frame()

NumberOfGuestsWeeklyResort = dataResort['ukupno_gostiju'].groupby(dataResort['datum_dolaska']).sum()
NumberOfGuestsWeeklyResort = NumberOfGuestsWeeklyResort.resample('w').sum().to_frame()

NumberOfGuestsMonthlyResort = dataResort['ukupno_gostiju'].groupby(dataResort['datum_dolaska']).sum()
NumberOfGuestsMonthlyResort = NumberOfGuestsMonthlyResort.resample('m').sum().to_frame()

NumberOfGuestsYearlyResort = dataResort['ukupno_gostiju'].groupby(dataResort['datum_dolaska']).sum()
NumberOfGuestsYearlyResort = NumberOfGuestsYearlyResort.resample('y').sum().to_frame()

NumberOfGuestsMonthlyResort.head()

In [ ]:
# Ciljne varijable - hotel 1
NumberOfGuestsCity = dataCity[['datum_dolaska', 'ukupno_gostiju']]

NumberOfGuestsDailyCity = dataCity['ukupno_gostiju'].groupby(dataCity['datum_dolaska']).sum()
NumberOfGuestsDailyCity = NumberOfGuestsDailyCity.resample('d').sum().to_frame()

NumberOfGuestsWeeklyCity = dataCity['ukupno_gostiju'].groupby(dataCity['datum_dolaska']).sum()
NumberOfGuestsWeeklyCity = NumberOfGuestsWeeklyCity.resample('w').sum().to_frame()

NumberOfGuestsMonthlyCity = dataCity['ukupno_gostiju'].groupby(dataCity['datum_dolaska']).sum()
NumberOfGuestsMonthlyCity = NumberOfGuestsMonthlyCity.resample('m').sum().to_frame()

NumberOfGuestsYearlyCity = dataCity['ukupno_gostiju'].groupby(dataCity['datum_dolaska']).sum()
NumberOfGuestsYearlyCity = NumberOfGuestsYearlyCity.resample('y').sum().to_frame()

NumberOfGuestsMonthlyCity.head()

## STL dekompozicija

### Hotel 0

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(NumberOfGuestsDailyResort[:30])
plt.show()

dodat komentar vezan uz t.window, s.window, robust i periodicnost (7 - daily data with a weekly pattern)
red velicine na y osi

In [ ]:
stl = STL(NumberOfGuestsDailyResort, period=7, robust=True)
res = stl.fit()
fig = res.plot()
fig.set_figwidth(20)
fig.set_figheight(10)

In [ ]:
stl = STL(NumberOfGuestsDailyResort, period=7, trend=31, seasonal=365, robust=True)
res = stl.fit()
fig = res.plot()
fig.set_figwidth(20)
fig.set_figheight(10)

In [ ]:
plt.figure(figsize=(20,2))
plt.plot(NumberOfGuestsWeeklyResort)
plt.show()

In [ ]:
stl = STL(NumberOfGuestsWeeklyResort, period=4, robust=True)
res = stl.fit()
fig = res.plot()
fig.set_figwidth(20)
fig.set_figheight(10)

In [ ]:
stl = STL(NumberOfGuestsMonthlyResort, period=3, robust=True)
res = stl.fit()
fig = res.plot()
fig.set_figwidth(20)
fig.set_figheight(10)